In [1]:
import sys
sys.path.append("../src")
import numpy as np
import helpers
import implementations
import fitters

In [2]:
x_train, x_test, y_train, train_ids, test_ids = helpers.load_csv_data("../data/raw/")

In [3]:
from preprocessing import undefined_to_median, undefined_to_mean, prune_undefined

indices = [26,27,28,30,32,33,39,42,43,44,46,48,50,80,232,234,237,238,242,246,253,259,264,278,279,284,288]
x_train = x_train[:, indices]
x_test = x_test[:, indices]

In [4]:
def change_x_by_nan(feature, to_eleminate, to_replace=None):
    for v in to_eleminate: 
        feature[feature == v] = np.nan
    if to_replace is not None:
        for key, value in to_replace.items():
            feature[feature == key] = value
    return feature

for x in [x_train, x_test]:
    change_x_by_nan(x[:, 0], [7, 9, 88]) # GENHEALT
    change_x_by_nan(x[:, 1], [77, 99], {88: 0}) # PHYSHEAT
    change_x_by_nan(x[:, 2], [77, 99], {88: 0}) # MENTALHEALTH
    change_x_by_nan(x[:, 3],  [7, 9], {2: 0}) # HLTHPLN1
    change_x_by_nan(x[:, 4],  [7, 9], {2: 0}) # MEDCOST1
    change_x_by_nan(x[:, 5],  [7, 9], {8: 5}) # CHECKUP1
    change_x_by_nan(x[:, 6],  [7, 9], {2: 0}) # CVDSTRK3
    change_x_by_nan(x[:, 7],  [7, 9], {2: 0}) # CHCSCNCR
    change_x_by_nan(x[:, 8],  [7, 9], {2: 0}) # CHCOCNCR
    change_x_by_nan(x[:, 9],  [7, 9], {2: 0}) # CHCCOPD1
    change_x_by_nan(x[:, 10], [7, 9], {2: 0}) # ADDEPEV2
    change_x_by_nan(x[:, 23], [9], {2:0}) # _FRTLT1 
    change_x_by_nan(x[:, 24], [9], {2:0}) # _VEGLT1 
    change_x_by_nan(x[:, 25], [9], {2:0}) # _TOTINDA 
    change_x_by_nan(x[:, 11],  [7, 9], {2:0, 3:0, 1:2, 4:1}) # DIABETE3 ASK ELISA
    #change_x_by_nan(x[:, 12], [7, 9], {2: 0}) # SEX ASK ELISA (Nothing to change)
    change_x_by_nan(x[:, 13], [7, 9]) # MAXDRNKS
    change_x_by_nan(x[:, 14], [9], {1:0, 2:1}) # _RFHYPE5
    change_x_by_nan(x[:, 15], [9], {1:0, 2:1}) # _RFCHOL
    change_x_by_nan(x[:, 16], [9], {3:0, 2:1, 1:2}) # _ASTHMS1 
    change_x_by_nan(x[:, 17], [], {2:0}) # _DRDXAR1 
    change_x_by_nan(x[:, 19], [14]) # _AGEG5YR 
    change_x_by_nan(x[:, 20], []) # _BMI5 
    change_x_by_nan(x[:, 18], [9]) # _RACE 
    change_x_by_nan(x[:, 21], [9]) # _SMOKER3 
    change_x_by_nan(x[:, 22], [99900]) # _DRNKWEK 
    change_x_by_nan(x[:, 26], [99900]) # FC60_ 

In [5]:
## REVISE THIS 
def nan_to_mean(x):
    mean = np.nanmean(x, axis=0)
    ids = np.where(np.isnan(x))
    x[ids] = np.take(mean, ids[1])
    
nan_to_mean(x_train)
nan_to_mean(x_test)
assert(not np.isnan(x_train).any())
assert(not np.isnan(x_test).any())

In [6]:
gm = fitters.GradientFitter(y_train, x_train, 5000, 0.000001, 0.2)
w, loss, (acc_tr, acc_te) = gm.fit()

Train-Validate: Training error=44.27313622336532, Training accuracy=0.9110427110792814
Train-Validate: Test error=44.013637453809935, Test accuracy=0.9110427110792814


In [7]:
lsq = fitters.LeastSquareFitter(y_train, x_train, 0.2)
w, loss, (acc_tr, acc_te) = lsq.fit()
w, loss, (acc_tr, acc_te)

Train-Validate: Training error=0.14304229174225458, Training accuracy=0.9110122358175751
Train-Validate: Test error=0.14219718105983845, Test accuracy=0.9110122358175751


In [9]:
rrf = fitters.RidgeRegressionFitter(y_train, x_train, 0.0001, 0.2)
w, loss, (acc_tr, acc_te) = rrf.fit()

Train-Validate: Training error=0.14304238084515233, Training accuracy=0.9110122358175751
Train-Validate: Test error=0.5332857566454976, Test accuracy=0.9110122358175751


In [16]:
# Use scikit to see 
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier().fit(x_train, y_train)
clf.score(x_train, y_train)

y_pred = clf.predict(x_test)

helpers.create_csv_submission(test_ids, y_pred, "RC.csv")

Random Oversampling: Randomly duplicate examples in the minority class.
Random Undersampling: Randomly delete examples in the majority class.